# AOI02-Training Deep Learning Model

## Step 1: Read images and labels

In [3]:
import pandas as pd
#labes of train dataset
df = pd.read_csv('../Data/train.csv')
print(df.shape)
train_labels = df.iloc[:,1].values
print(train_labels)

(2528, 2)
[0 1 1 ... 3 0 5]


In [13]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.python.keras.applications.imagenet_utils import preprocess_input
#Images of train dataset
maxTrainNum = 2528
#Assign a list for storing image data
img_data_list=[]
for i in range(0,maxTrainNum):
    imgname="../Data/train_images/train_{0:0>5d}_224.png".format(i)#first image as train_00000_224.png
    img = image.load_img(imgname)
    x=image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    img_data_list.append(x)

## Step 2: Convert image to array

In [6]:
img_data = np.array(img_data_list)
print (img_data.shape)  #(2528, 1, 224, 224, 3)
img_data=np.rollaxis(img_data,1,0)
print (img_data.shape)  #(1, 2528, 224, 224, 3)
img_data=img_data[0]
print (img_data.shape)  #(2528, 224, 224, 3)

(2528, 1, 224, 224, 3)
(1, 2528, 224, 224, 3)
(2528, 224, 224, 3)


## Step 3: Configure a customized VGG-16 Deep Learning Model

In [7]:
import os
from tensorflow.python.keras import utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

In [8]:
num_classes = 6
#input shape
###  CODE HERE ###  (≈ 1 lines)
image_input = Input(shape=(224, 224, 3))

#Use the VGG16 model 
###  CODE HERE ###  (≈ 1 lines)
model = VGG16(input_tensor=image_input, include_top=True,weights='imagenet')

#Summary of the customize VGG16 model
###  CODE HERE ###  (≈ 1 lines)
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
bloc

In [9]:
last_layer = model.get_layer('fc2').output
out = Dense(num_classes, activation='softmax', name='output')(last_layer)
custom_vgg_model = Model(image_input, out)
custom_vgg_model.summary()

for layer in custom_vgg_model.layers[:-1]:
    layer.trainable = False

#Set the model's loss function, optimizer, and metrics
###  CODE HERE ###  (≈ 1 lines)
custom_vgg_model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

## Step 4: Traing the customized model

In [10]:
X_train= img_data
# one-hot encoding
y_train = utils.np_utils.to_categorical(train_labels, num_classes)

In [12]:
import time
t=time.time()
#The parameters for the training model
###  CODE HERE ###  (≈ 1 lines)
hist = custom_vgg_model.fit(X_train, y_train, batch_size=32, epochs=10, verbose=1)
print('Training time: %s' % time.time())

Epoch 1/10


ResourceExhaustedError: OOM when allocating tensor with shape[32,64,224,224] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[{{node block1_conv2/Conv2D}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[{{node loss/mul}}]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


## Step 5: Save the customized model

In [ ]:
custom_vgg_model.save("myVgg16.h5")

In [ ]:
!ls